<a href="https://colab.research.google.com/github/mausam3407/Application/blob/main/cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf

In [2]:
import zipfile
 
zip_file = "/content/drive/MyDrive/Kaggle/cat-breeds-dataset.zip"
 
try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall()
        print("Extracted all")
except:
    print("Invalid file")

Extracted all


In [3]:
data = ['Bombay', 'Calico', 'Burmese', 'Himalayan','Munchkin', 'Ragdoll', 'Siberian', 'British Shorthair', 'Russian Blue', 'Dilute Calico']


In [4]:
import os
os.mkdir(f"{os.getcwd()}/temp")
os.mkdir(f"{os.getcwd()}/temp/training/")
os.mkdir(f"{os.getcwd()}/temp/validation/")
os.mkdir(f"{os.getcwd()}/temp/testing/")

In [5]:
for d in data:
    os.mkdir(f"{os.getcwd()}/temp/training/{d}")
    os.mkdir(f"{os.getcwd()}/temp/validation/{d}")
    os.mkdir(f"{os.getcwd()}/temp/testing/{d}")

In [6]:
def split_data(SOURCE, TRAINING,VALIDATION, TESTING):
    files = os.listdir(SOURCE)
    files = random.sample(files,len(files))
    k = 0
    for file in files:
        k=k+1
        source = os.path.join(SOURCE,file)
        dest_tr = os.path.join(TRAINING,file)
        dest_tt = os.path.join(TESTING,file)
        dest_tv = os.path.join(VALIDATION,file)
        if os.path.getsize(source)!=0:
            if (k<=0.8*(len(files))):
                copyfile(source,dest_tr)
            elif (k<=0.9*(len(files))):
                copyfile(source,dest_tt)
            else:
                copyfile(source,dest_tv)

In [7]:
import random
from shutil import copyfile
for d in data:
    split_data(f"{os.getcwd()}/images/{d}",f"{os.getcwd()}/temp/training/{d}",f"{os.getcwd()}/temp/validation/{d}",f"{os.getcwd()}/temp/testing/{d}")
    

In [31]:
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = f"{os.getcwd()}/temp/training/"
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

VALIDATION_DIR = f"{os.getcwd()}/temp/validation/"
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

TEST_DIR = f"{os.getcwd()}/temp/testing/"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

Found 12520 images belonging to 10 classes.
Found 1569 images belonging to 10 classes.
Found 1565 images belonging to 10 classes.


In [15]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights='imagenet')

last_layer = model.get_layer('mixed10')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(model.input, x)


last layer output shape:  (None, 3, 3, 2048)


In [18]:
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit(train_generator,
                              epochs=20,
                              verbose=1,
                   validation_data=validation_generator,
                    callbacks=[callback])

Epoch 1/20
196/196 [==============================] - 119s 546ms/step - loss: 1.4412 - acc: 0.5260 - val_loss: 1.0228 - val_acc: 0.6807
Epoch 2/20
196/196 [==============================] - 105s 535ms/step - loss: 0.9671 - acc: 0.6601 - val_loss: 0.9875 - val_acc: 0.6864
Epoch 3/20
196/196 [==============================] - 104s 530ms/step - loss: 0.8495 - acc: 0.7043 - val_loss: 1.0876 - val_acc: 0.6979
Epoch 4/20
196/196 [==============================] - 103s 526ms/step - loss: 0.7787 - acc: 0.7320 - val_loss: 1.0257 - val_acc: 0.7055
Epoch 5/20
196/196 [==============================] - 105s 535ms/step - loss: 0.7255 - acc: 0.7496 - val_loss: 0.8855 - val_acc: 0.7215
Epoch 6/20
196/196 [==============================] - 103s 524ms/step - loss: 0.6727 - acc: 0.7660 - val_loss: 0.9794 - val_acc: 0.7298
Epoch 7/20
196/196 [==============================] - 105s 534ms/step - loss: 0.6386 - acc: 0.7749 - val_loss: 1.1816 - val_acc: 0.7100
Epoch 8/20
196/196 [============================

In [20]:
model.save('/content/drive/MyDrive/Kaggle/cat_model.h5')

In [65]:
labels = test_generator.labels
test_generator.shuffle = False
pred = model.predict(test_generator)

In [68]:
m = tf.keras.metrics.Accuracy()
m.update_state(pred.argmax(axis=1),labels)
print(f"Accuray: {m.result().numpy()}")

Accuray: 0.7015974521636963


In [73]:
from sklearn.metrics import f1_score
print(f"f1_score: {f1_score(labels,pred.argmax(axis=1),average='weighted')}")

f1_score: 0.6870487733790145


In [83]:
from sklearn.metrics import confusion_matrix
print(f"Confusion Matrix: \n{confusion_matrix(labels,pred.argmax(axis=1))}")

Confusion Matrix: 
[[173   0   0   3   0   1   0   1   5   0]
 [  7  17   0   5   8   0   2   8  10   0]
 [ 10   0   8   1   1   2   0  10   2   0]
 [  2   1   0 271  53   2   0   9   9   0]
 [  0   7   0  68 184   1   1  25  37   0]
 [  2   1   1   2   1  68   0  54   1   0]
 [  4   1   2   1   5   0   1   4   0   0]
 [  4   8   3  13   4  31   0 201   3   0]
 [  2   4   0   2   4   0   0   3 172   0]
 [  0   0   1   1   1   1   0  11   1   3]]
